# Module 0

Implement the forward pass and backward pass for a small NN that computers for the XOR problem

For now, I just left it at the forward pass and one backward pass (not topo-sorting and doing them all)

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from micrograd import nn
import random

In [9]:
class Value:
    def __init__(self, data, grad = 0, _op = ''):
        self.data = data
        self.grad = grad
        self.op = _op
        self.backward = lambda: None
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, _op='+')
        
        def _backward():
            self.grad += out.grad  # Distribute gradient; use +=
            other.grad += out.grad # Distribute gradient; use +=
        out.backward = _backward
        
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, _op='*')
        
        def _backward():
            # Chain rule for multiplication; use +=
            self.grad += other.data * out.grad 
            other.grad += self.data * out.grad
        out.backward = _backward
        
        return out
        
    def tanh(self):
        t = math.tanh(self.data)
        out = Value(t, grad=0, _op='tanh')
        def _backward():
            self.grad += out.grad * (1 - out.data**2)
        out.backward = _backward
        return out
    
    
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    
    # x is a list of inputs
    def __call__(self, x):
        action_potential = sum((wi * xi for wi, xi in zip(self.w, x)), self.b) #need zip because i didn't define rhand sum
        return action_potential.tanh()
    
class Layer:
    def __init__(self,nin, nout):
        self.nodes = [Neuron(nin) for _ in range(nout)]
    def __call__(self, x):
        outs = [n(x) for n in self.nodes]
        return outs

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    
nn = MLP(2,[2,1])
nn([0,2])[0].data

-0.6642754889235848